In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [2]:
def timing(f):
    def wrap(*args, **kwargs):
        time1 = time.time()
        ret = f(*args, **kwargs)
        time2 = time.time()
        print('%s function took %0.3f ms' % (f.__name__, (time2-time1)*1000.0))
        return ret
    return wrap

In [3]:
def evaluate_rewards_and_transitions(problem, mutate=False):
    # Enumerate state and action space sizes
    num_states = problem.observation_space.n
    num_actions = problem.action_space.n

    # Initialize Transition and Reward matrices
    R = np.zeros((num_states, num_actions, num_states))
    T = np.zeros((num_states, num_actions, num_states))

    # Iterate over states, actions, and transitions
    for state in range(num_states):
        for action in range(num_actions):
            for transition in problem.env.P[state][action]:
                probability, next_state, reward, done = transition
                R[state, action, next_state] = reward
                T[state, action, next_state] = probability

            # Normalize T across state + action axes
            T[state, action,:] /= np.sum(T[state, action,:])

    # Conditionally mutate and return
    if mutate:
        problem.env.R = R
        problem.env.T = T
    return R, T

In [4]:
@timing
def value_iteration(problem, R=None, T=None, gamma=0.9, max_iterations=10**6, delta=10**-3):
    """ Runs Value Iteration on a gym problem """
    value_fn = np.zeros(problem.observation_space.n)
    if R is None or T is None:
        R, T = evaluate_rewards_and_transitions(problem)

    for i in range(max_iterations):
        previous_value_fn = value_fn.copy()
        Q = np.einsum('ijk,ijk -> ij', T, R + gamma * value_fn)
        value_fn = np.max(Q, axis=1)

        if np.max(np.abs(value_fn - previous_value_fn)) < delta:
            break

    # Get and return optimal policy
    policy = np.argmax(Q, axis=1)
    return policy, i + 1

In [5]:
def encode_policy(policy, shape):
    """ One-hot encodes a policy """
    encoded_policy = np.zeros(shape)
    encoded_policy[np.arange(shape[0]), policy] = 1
    return encoded_policy

In [6]:
@timing
def policy_iteration(problem, R=None, T=None, gamma=0.9, max_iterations=10**6, delta=10**-3):
    """ Runs Policy Iteration on a gym problem """
    num_spaces = problem.observation_space.n
    num_actions = problem.action_space.n

    # Initialize with a random policy and initial value function
    policy = np.array([problem.action_space.sample() for _ in range(num_spaces)])
    value_fn = np.zeros(num_spaces)

    # Get transitions and rewards
    if R is None or T is None:
        R, T = evaluate_rewards_and_transitions(problem)

    # Iterate and improve policies
    for i in range(max_iterations):
        previous_policy = policy.copy()

        for j in range(max_iterations):
            previous_value_fn = value_fn.copy()
            Q = np.einsum('ijk,ijk -> ij', T, R + gamma * value_fn)
            value_fn = np.sum(encode_policy(policy, (num_spaces, num_actions)) * Q, 1)

            if np.max(np.abs(previous_value_fn - value_fn)) < delta:
                break

        Q = np.einsum('ijk,ijk -> ij', T, R + gamma * value_fn)
        policy = np.argmax(Q, axis=1)

        if np.array_equal(policy, previous_policy):
            break

    # Return optimal policy
    return policy, i + 1

In [7]:
def print_policy(policy, mapping=None, shape=(0,)):
    print(np.array([mapping[action] for action in policy]).reshape(shape))

In [139]:
def run_discrete(environment_name, mapping, shape=None , tol=10**-3, env = None):   
    if env == None:
        problem = gym.make(environment_name)
    else:
        from gym.wrappers.time_limit import TimeLimit
        problem = TimeLimit(env)

    print('== {} =='.format(environment_name))
    print('Actions:', problem.env.action_space.n)
    print('States:', problem.env.observation_space.n)
    #print(problem.env.desc)
    print()

    print('== Value Iteration ==')
    value_policy, iters = value_iteration(problem, delta = tol)
    print('Iterations:', iters)
    print()

    if shape is not None:
        print('== Value Iterated Policy ==')
        print_policy(value_policy, mapping, shape)
        print()


    print('== Policy Iteration ==')
    policy, iters = policy_iteration(problem, delta = tol)
    print('Iterations:', iters)
    print()

    if shape is not None:
        print('== Policy Iterated Policy ==')
        print_policy(policy, mapping, shape)
        print()

    diff = sum([abs(x-y) for x, y in zip(policy.flatten(), value_policy.flatten())])
    print('Discrepancy:', diff)
    print()

    

    return policy , value_policy


In [145]:
def test_policy(policy, env_name = None, env = None):
    if env == None:
        problem = gym.make(env_name)
    else:
        from gym.wrappers.time_limit import TimeLimit
        problem = TimeLimit(env)
    
    
    done = False
    total_reward = 0
    num_steps = []
    for i in range(1000):
        state = problem.reset()
        done = False
        i = 0
        while not done:
            #print ("state is :{} ".format(state))
            #print (agent.Q[state])

            action = policy[state]
            #print ("action is :{}".format(action))

            state , reward, done, _ = problem.step(action)
            i +=1
            #problem.render()
        #problem.render()
        total_reward += reward
        num_steps.append(i)

    print ("Total Reward: {}".format(total_reward))
    print ("Average Steps: {}".format(np.mean(num_steps)))

In [109]:
mapping = {0: "L", 1: "D", 2: "R", 3: "U"}
shape = (4, 4)
frozen_policy, frozen_value_pol = run_discrete('FrozenLake-v0', mapping, shape = None, tol=10**-10)

[2018-04-14 17:07:31,802] Making new env: FrozenLake-v0


== FrozenLake-v0 ==
Actions: 4
States: 16

== Value Iteration ==
value_iteration function took 9.016 ms
Iterations: 142

== Policy Iteration ==
policy_iteration function took 16.540 ms
Iterations: 2

Discrepancy: 0



In [110]:
test_policy(frozen_policy,'FrozenLake-v0' )

[2018-04-14 17:07:33,704] Making new env: FrozenLake-v0


Total Reward: 714.0
Average Steps: 41.841


In [146]:
test_policy(frozen_value_pol,'FrozenLake-v0' )

[2018-04-14 17:25:09,477] Making new env: FrozenLake-v0


Total Reward: 754.0
Average Steps: 40.181


In [112]:
# FROZEN LAKE LARGE
shape = (8, 8)
froz88_pol , froz88_val_pol = run_discrete('FrozenLake8x8-v0', mapping, shape = None, tol = 10**-5)


[2018-04-14 17:07:36,624] Making new env: FrozenLake8x8-v0


== FrozenLake8x8-v0 ==
Actions: 4
States: 64

== Value Iteration ==
value_iteration function took 8.706 ms
Iterations: 66

== Policy Iteration ==
policy_iteration function took 25.017 ms
Iterations: 9

Discrepancy: 0



In [113]:
test_policy(froz88_pol,'FrozenLake8x8-v0' )

[2018-04-14 17:07:38,567] Making new env: FrozenLake8x8-v0


Total Reward: 716.0
Average Steps: 70.681


In [114]:
test_policy(froz88_val_pol,'FrozenLake8x8-v0' )

[2018-04-14 17:07:39,424] Making new env: FrozenLake8x8-v0


Total Reward: 775.0
Average Steps: 71.392


In [115]:
def test_policy_Taxi(policy, env_name):
    problem = gym.make(env_name)
    done = False
    total_rewards = []
    num_steps = []
    for i in range(1000):
        state = problem.reset()
        done = False
        i = 0
        ep_reward = 0
        while not done:
            #print ("state is :{} ".format(state))
            #print (agent.Q[state])

            action = policy[state]
            #print ("action is :{}".format(action))

            state , reward, done, _ = problem.step(action)
            i +=1
            ep_reward+=reward
            #problem.render()
        #problem.render()
        total_rewards.append(ep_reward)
        num_steps.append(i)

    print ("Average Episode Reward: {}".format(np.mean(total_rewards)))
    print ("Average Steps: {}".format(np.mean(num_steps)))

In [116]:
mapping = {0: "S", 1: "N", 2: "E", 3: "W", 4: "P", 5: "D"}
tax_pol, tax_val_pol = run_discrete('Taxi-v2', mapping, tol = 10**-4)

[2018-04-14 17:07:40,889] Making new env: Taxi-v2


== Taxi-v2 ==
Actions: 6
States: 500

== Value Iteration ==
value_iteration function took 870.393 ms
Iterations: 117

== Policy Iteration ==
policy_iteration function took 2129.027 ms
Iterations: 16

Discrepancy: 0



In [117]:
test_policy_Taxi(tax_pol, 'Taxi-v2')

[2018-04-14 17:07:45,112] Making new env: Taxi-v2


Average Episode Reward: 8.558
Average Steps: 12.442


In [118]:
test_policy_Taxi(tax_val_pol, 'Taxi-v2')

[2018-04-14 17:07:45,488] Making new env: Taxi-v2


Average Episode Reward: 8.486
Average Steps: 12.514


In [136]:
from gym.envs.toy_text import FrozenLakeEnv
# NEW 16x16 FROZEN LAKE
desc = ["SFFFFFFFHFFFFFFF",
        "FFFFFFFFFFFFFFFF",
        "FFFHFFFFFFFHFFFF",
        "FFFFFHFFFFFFFHFF",
        "FFFHFFFFFFFHFFFF",
        "FHHFFFHFFHHFFFHF",
        "FHFFHFHFFHFFHFHF",
        "FFFHFFFFFFFHFFFH",
        "HFFFFFFFHFFFFFFF",
        "FFFFFFHHHFFFFFFF",
        "FFFHFFFFFFFHFFFF",
        "FFFFFHFFFFFFFHFF",
        "FFFHFFFFFFFHFFFF",
        "FHHFFFHFFHHFFFHF",
        "FHFFHFHFFHFFHFHF",
        "FFFHFFFFFFFHFFFG"]
frozen16env = FrozenLakeEnv(desc=desc, map_name=None)

In [155]:
## 16x16 Lake!
mapping = {0: "L", 1: "D", 2: "R", 3: "U"}
shape = (16, 16)
froz16_pol, froz16_val_pol = run_discrete("FrozenLake16x16v0", mapping, shape = None, env = frozen16env, tol = 10**-10)

== FrozenLake16x16v0 ==
Actions: 4
States: 256

== Value Iteration ==
value_iteration function took 157.778 ms
Iterations: 153

== Policy Iteration ==
policy_iteration function took 472.609 ms
Iterations: 7

Discrepancy: 0



In [156]:
test_policy(froz16_pol, env = frozen16env )

Total Reward: 151.0
Average Steps: 112.596


In [157]:
test_policy(froz16_val_pol, env=frozen16env)

Total Reward: 112.0
Average Steps: 110.059
